In [2]:
import requests
import pandas as pd

In [3]:
# http://api.worldweatheronline.com/premium/v1/past-weather.ashx?key=6f2d4be584aa413f9df230512210306&q=45.53,87.38&date=2008-09-08&tp=1&format=json
# above is example

In [5]:
#import cleaned NFL schedule
schedule_csv = "NFL_Schedule.csv"

schedule = pd.read_csv(schedule_csv)
schedule.head()

,Full Date,New Time,Week,City,Home Team,Winner
0,2008-09-04,7,1,New York,Giants,Giants
1,2008-09-07,1,1,Philadelphia,Eagles,Eagles
2,2008-09-07,1,1,Miami,Dolphins,Jets
3,2008-09-07,1,1,New England,Patriots,Patriots
4,2008-09-07,1,1,Pittsburgh,Steelers,Steelers
